In [2]:
# iplot/cufflinks

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import calendar
from matplotlib.pyplot import subplots, show
import plotly.plotly as py
import pandas as pd

pd.options.mode.chained_assignment = None

%matplotlib inline

In [3]:
weatherdata = pd.read_csv('detail_trim2.csv', encoding = "ISO-8859-1")

weatherdata['EPISODE_ID'].nunique()
weatherdata['EVENT_ID'].nunique()

del weatherdata['YEAR']
del weatherdata['MONTH_NAME']
del weatherdata['BEGIN_DAY']
del weatherdata['BEGIN_TIME']
print(weatherdata.EVENT_REDUCED.unique())
weatherdata.tail()

['Tornado' 'Hail' 'Wind' 'Flood' 'Winter Storm' 'Winter Weather'
 'Debris Flow' 'Lightning' 'Heat' 'Rain' 'Wildfire/Smoke' 'Waves/Surge'
 'Other' 'Drought' 'Tropical Storm']


,BEGIN_DATE_TIME,END_DATE_TIME,EPISODE_ID,EVENT_ID,STATE,CZ_TYPE,EVENT_TYPE,EVENT_REDUCED,CZ_TIMEZONE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,BEGIN_LAT,BEGIN_LON
1471783,21-AUG-17 10:48:00,21-AUG-17 10:48:00,120083.0,719531,IOWA,C,Hail,Hail,CST-6,0,0,0,0,0.0,0.0,43.4300,-96.1700
1471784,21-AUG-17 10:57:00,21-AUG-17 10:57:00,120083.0,719533,IOWA,C,Hail,Hail,CST-6,0,0,0,0,0.0,0.0,43.4000,-95.7300
1471785,12-AUG-17 16:25:00,12-AUG-17 16:35:00,120278.0,720673,CALIFORNIA,C,Thunderstorm Wind,Wind,PST-8,0,0,0,0,15000.0,0.0,32.9156,-116.2443
1471786,12-AUG-17 16:30:00,12-AUG-17 17:30:00,120278.0,720675,CALIFORNIA,C,Flash Flood,Flood,PST-8,0,0,0,0,5000.0,0.0,32.9181,-116.2540
1471787,12-AUG-17 16:10:00,12-AUG-17 18:00:00,120278.0,720679,CALIFORNIA,C,Flash Flood,Flood,PST-8,0,0,0,0,5000.0,0.0,33.0425,-116.1047


In [4]:
nevada = weatherdata.loc[weatherdata['STATE'] == 'NEVADA']
nevada['EVENT_TYPE'].unique()

array(['Hail', 'Thunderstorm Wind', 'Tornado', 'HAIL FLOODING',
       'THUNDERSTORM WINDS/FLASH FLOOD', 'THUNDERSTORM WINDS FUNNEL CLOU',
       'Heavy Snow', 'High Wind', 'Dust Devil', 'Lightning',
       'Flash Flood', 'Funnel Cloud', 'Heavy Rain', 'Flood',
       'Winter Storm', 'Dust Storm', 'Cold/Wind Chill', 'Wildfire',
       'Dense Fog', 'Avalanche', 'Blizzard', 'Heat',
       'Hurricane (Typhoon)', 'Winter Weather', 'Strong Wind',
       'Ice Storm', 'Excessive Heat', 'Freezing Fog', 'Frost/Freeze',
       'Landslide', 'Lake-Effect Snow', 'Debris Flow',
       'Extreme Cold/Wind Chill'], dtype=object)

In [5]:
us_state_abbrev = {'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR','California': 'CA','Colorado': 'CO','Connecticut': 'CT',
'Delaware': 'DE','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA',
'Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN',
'Mississippi': 'MS','Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ',
'New Mexico': 'NM','New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR',
'Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX',
'Utah': 'UT','Vermont': 'VT','Virginia': 'VA','Washington': 'WA','West Virginia': 'WV', 'Wisconsin': 'WI','Wyoming': 'WY'}

weatherdata['STATE'] = weatherdata['STATE'].str.lower().str.title()
weatherdata['STATE'] = weatherdata['STATE'].replace(us_state_abbrev, regex=True)

In [6]:
def val_sort(df,ser,val,clist):
    valdf = df.loc[df[ser] == val,]   
    valdf['COUNT'] = 1
    
    for column in valdf:
        if column in clist:
            del valdf[column]
             
    valdf = valdf.groupby(["STATE"]).sum()
    valdf = valdf.reset_index()
    
    return valdf

def map_us(series1,series2,maptitle,bartitle,color):

    scl = [[0.0, 'rgb(242,240,247)'],[1.0, color]]

    data = [ dict(
            type='choropleth',
            colorscale = scl,
            autocolorscale = False,
            locations = series1,
            z = series2.astype(float),
            locationmode = 'USA-states',
            marker = dict(
                line = dict (
                    color = 'rgb(255,255,255)',
                    width = 2
                ) ),
            colorbar = dict(
                title = bartitle)
            ) ]

    layout = dict(
            title = maptitle,
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showlakes = True,
                lakecolor = 'rgb(255, 255, 255)'),
                 )

    fig = dict( data=data, layout=layout )
    
    return fig

In [7]:
cols = ['BEGIN_DATE_TIME', 'END_DATE_TIME','CZ_TYPE', 'EVENT_TYPE', 'EPISODE_ID','EVENT_ID','EVENT_REDUCED', 'CZ_TIMEZONE', 'INJURIES_DIRECT',
       'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT','DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'BEGIN_LAT', 'BEGIN_LON']

In [8]:
hailweather = val_sort(weatherdata,'EVENT_REDUCED','Hail',cols)
hailweather

,STATE,COUNT
0,AK,23
1,AL,7041
2,AR,10729
3,AZ,1008
4,Atlantic North,103
5,Atlantic South,43
6,CA,689
7,CO,11593
8,CT,611
9,DE,115


In [1]:
fig = map_us(rainweather['STATE'],rainweather['COUNT'],'Rain Events per State (1950-2017)','Event Total Count', 'rgb(39, 11, 224)')
py.iplot(fig, filename='d3-cloropleth-map' )

NameError: name 'val_sort' is not defined

In [203]:
rainweather = val_sort(weatherdata,'EVENT_REDUCED','Rain',cols)
fig = map_us(hailweather['STATE'],hailweather['COUNT'],'Hail Events per State (1950-2017)','Event Total Count','rgb(229, 41, 216)')
py.iplot(fig, filename='d3-cloropleth-map' )

In [204]:
fireweather = val_sort(weatherdata,'EVENT_REDUCED','Wildfire/Smoke',cols)
fig = map_us(fireweather['STATE'],fireweather['COUNT'],'Fire Events per State (1950-2017)','Event Total Count','rgb(229, 121, 13)')
py.iplot(fig, filename='d3-cloropleth-map' )

In [205]:
surgeweather = val_sort(weatherdata,'EVENT_REDUCED','Waves/Surge',cols)
fig = map_us(fireweather['STATE'],fireweather['COUNT'],'Surge Events per State (1950-2017)','Event Total Count','rgb(0, 247, 189)')
py.iplot(fig, filename='d3-cloropleth-map' )

In [210]:
droughtweather = val_sort(weatherdata,'EVENT_REDUCED','Drought',cols)
fig = map_us(droughtweather['STATE'],droughtweather['COUNT'],'Drought Events per State (1950-2017)','Event Total Count','rgb(96, 57, 33)')
py.iplot(fig, filename='d3-cloropleth-map' )

In [208]:
nevada = weatherdata.loc[weatherdata['STATE'] == 'Nevada']
nevada['EVENT_TYPE'].unique()

In [218]:
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure


longitude  = [44.990961]
latitude = [41.552164]

source = ColumnDataSource(data=dict(longitude=longitude, latitude=latitude))

p = figure(plot_width=400, plot_height=400)
p.circle(x='longitude', y='latitude', source=source)
show(p)